In [1]:
#https://www.analyticsvidhya.com/blog/2017/09/understaing-support-vector-machine-example-code/
#http://scikit-learn.org/stable/modules/svm.html

In [2]:
#import libraries
from sklearn import svm,datasets
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import io
import zipfile
import requests
import geopandas as gpd
import shapely.geometry
from shapely.geometry import Point
import pyproj
import math
import os, sys

from skimage.feature import hog
from skimage import data, exposure

In [3]:
path = '..//..//..//Documents//Test_sets//Testing_Sample//HOG_testsets_10m//comp//'

In [4]:
dataset = pd.read_csv(path+"Meanfinal//Mean10m_o4p5b1.csv")

In [5]:
dataset = dataset.drop(['Unnamed: 0'], axis=1)
# dataset = dataset.astype(float)
len(dataset)

2710

In machine learning, support vector machines (SVMs, also support vector networks[1]) are supervised learning models with associated learning algorithms that analyze data and recognize patterns, used for classification and regression analysis. The basic SVM takes a set of input data and predicts, for each given input, which of two possible classes forms the output, making it a non-probabilistic binary linear classifier. Given a set of training examples, each marked as belonging to one of two categories, an SVM training algorithm builds a model that assigns new examples into one category or the other. An SVM model is a representation of the examples as points in space, mapped so that the examples of the separate categories are divided by a clear gap that is as wide as possible. New examples are then mapped into that same space and predicted to belong to a category based on which side of the gap they fall on. In addition to performing linear classification, SVMs can efficiently perform non-linear classification using what is called the kernel trick, implicitly mapping their inputs into high-dimensional feature spaces.

The logit model we just implemented was great in that it showed exactly where to draw our decision boundary or our 'survival cut off'. But if you’re like me, you could have thought, "So uhhh, what if a straight line just doesn’t cut it". A linear line is okay, but can we do better? Perhaps a more complex decision boundary like a wave, circle, or maybe some sort of strange polygon would describe the variance observed in our sample better than a line. Imagine if we were predicating survival based on age. It could be a linear decision boundary, meaning each additional time you've gone around the sun you were 1 unit more or less likely to survive. But I think it could be easy to imagine some sort of curve, where a young healthy person would have the best chance of survival, and sadly the very old and very young a like: a poor chance. Now that’s a interesting question to answer. But our logit model can only evaluate a linear decision boundary. How do we get around this? With the usual answer to life the universe and everything;  $MATH$.

In [6]:
from sklearn.model_selection import cross_val_score, KFold
target = dataset['Selection']
data = dataset.drop(['Selection'], axis=1)

print(data.head())
print(target.head())

          A        B1        B2        B3        B4        B5        B6  \
0  0.426813  0.653130  0.318596  0.530206  0.742973  0.818164  0.619180   
1  0.932927  0.734760  0.719825  0.747291  0.670498  0.773859  0.421832   
2  0.668088  0.768380  0.599669  0.275217  0.659798  0.851904  0.838322   
3  0.875350  0.660110  0.999942  0.865506  0.153837  0.433640  0.797535   
4  0.894774  0.839883  0.964893  0.636252  0.038287  0.707062  0.980859   

         B7        B8        C1    ...          C11       C12       C13  \
0  0.760607  0.678575  0.365601    ...     0.000000  0.000000  0.000000   
1  0.484944  0.689244  0.690450    ...     0.864885  0.592226  0.576842   
2  0.297506  0.370237  0.881203    ...     0.689392  0.875435  0.699040   
3  0.695872  0.123011  0.723914    ...     0.998467  0.926277  0.858568   
4  0.317538  0.309171  0.987162    ...     0.936411  0.547121  0.269800   

        C14       C15       C16     mindep    cellav     allav    drange  
0  0.000000  0.000000  

In [7]:
dfList = dataset['Selection'].tolist()

In [8]:
from sklearn.model_selection import train_test_split

In [17]:
x_train, x_test, y_train, y_test = train_test_split(data, dfList, test_size=0.3, stratify = target,random_state=16)

In [21]:
clf = svm.SVC(kernel='rbf', C = 6, gamma = 0.04)

In [ ]:
{'C': 6, 'gamma': 0.03, 'kernel': 'rbf'}

In [22]:
clf.fit(x_train,y_train)

SVC(C=6, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.04, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [23]:
from sklearn.metrics import accuracy_score

predicted = clf.predict(x_test)
accuracy = accuracy_score (y_test,predicted)

# print(f'Out-of-bag score estimate: {clf.oob_score_:.3}')
print(f'Mean accuracy score: {accuracy:.3}')

Mean accuracy score: 0.685


In [13]:
x_train = x_train.astype(float)
# y_train = y_train.astype(float)
x_test = x_test.astype(float)
# y_test = y_test.astype(float)


In [14]:
from sklearn.model_selection import GridSearchCV 

In [15]:
from sklearn.model_selection import GridSearchCV 

parameters = {'kernel':('linear', 'rbf'), 'C':[1,2,3,4,5,6,7,8,9,10], 'gamma': 
              [0.01,0.02,0.03,0.04,0.05,0.10,0.2,0.3,0.4,0.5]}
svr = svm.SVC()
grid = GridSearchCV(svr, parameters)
grid.fit(x_train, y_train)
predicted = grid.predict(x_test)
cnf_matrix = confusion_matrix(y_test, predicted)
print(cnf_matrix)

NameError: name 'confusion_matrix' is not defined

In [16]:
grid.best_params_

{'C': 6, 'gamma': 0.03, 'kernel': 'rbf'}

In [16]:
tuned_parameters = {'kernel':('linear', 'rbf'), 'C':[1,2,3,4,5,6,7,8,9,10], 'gamma': 
              [0.01,0.02,0.03,0.04,0.05,0.10,0.2,0.3,0.4,0.5]}
scores = ['precision', 'recall']
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(svm.SVC(), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(x_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(x_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 3, 'gamma': 0.04, 'kernel': 'rbf'}

Grid scores on development set:

0.672 (+/-0.046) for {'C': 1, 'gamma': 0.01, 'kernel': 'linear'}
0.664 (+/-0.045) for {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
0.672 (+/-0.046) for {'C': 1, 'gamma': 0.02, 'kernel': 'linear'}
0.667 (+/-0.050) for {'C': 1, 'gamma': 0.02, 'kernel': 'rbf'}
0.672 (+/-0.046) for {'C': 1, 'gamma': 0.03, 'kernel': 'linear'}
0.668 (+/-0.050) for {'C': 1, 'gamma': 0.03, 'kernel': 'rbf'}
0.672 (+/-0.046) for {'C': 1, 'gamma': 0.04, 'kernel': 'linear'}
0.673 (+/-0.027) for {'C': 1, 'gamma': 0.04, 'kernel': 'rbf'}
0.672 (+/-0.046) for {'C': 1, 'gamma': 0.05, 'kernel': 'linear'}
0.675 (+/-0.040) for {'C': 1, 'gamma': 0.05, 'kernel': 'rbf'}
0.672 (+/-0.046) for {'C': 1, 'gamma': 0.1, 'kernel': 'linear'}
0.670 (+/-0.016) for {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
0.672 (+/-0.046) for {'C': 1, 'gamma': 0.2, 'kernel': 'linear'}
0.667 (+/-0.0

NameError: name 'X_test' is not defined

http://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html